In [1]:
import pandas
from sklearn import linear_model
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

In [2]:
df = pd.DataFrame(pd.read_csv("Price.csv"))

In [3]:
df['arrival_date'] = pd.to_datetime(df['arrival_date'],errors='coerce')

C:\Users\hkish\AppData\Local\Temp\ipykernel_580\136839367.py:1: UserWarning: Parsing '13-01-2019' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['arrival_date'] = pd.to_datetime(df['arrival_date'],errors='coerce')
C:\Users\hkish\AppData\Local\Temp\ipykernel_580\136839367.py:1: UserWarning: Parsing '14-01-2019' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['arrival_date'] = pd.to_datetime(df['arrival_date'],errors='coerce')
C:\Users\hkish\AppData\Local\Temp\ipykernel_580\136839367.py:1: UserWarning: Parsing '15-01-2019' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['arrival_date'] = pd.to_datetime(df['arrival_date'],errors='coerce')
C:\Users\hkish\AppData\Local\Temp\ipykernel_580\136839367.py:1: UserWarning: Parsing '16-01-2019' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent

In [4]:
column_1 = df['arrival_date']

In [5]:
db=pd.DataFrame({"year": column_1.dt.year,
              "month": column_1.dt.month,
              "day": column_1.dt.day,
             })

In [6]:
df['year'] = db['year']
df['month'] = db['month']
df['day'] = db['day']

In [7]:
df = df.drop('arrival_date',axis=1)

In [8]:
X = df[['commodity', 'year', 'month', 'day']]
y = df['modal_price']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [10]:
n_estimators = [5,20,50,100] # number of trees in the random forest
max_features = ['auto', 'sqrt'] # number of features in consideration at every split
max_depth = [int(x) for x in np.linspace(10, 120, num = 12)] # maximum number of levels allowed in each decision tree
min_samples_split = [2, 6, 10] # minimum sample number to split a node
min_samples_leaf = [1, 3, 4] # minimum sample number that can be stored in a leaf node
bootstrap = [True, False] # method used to sample data points

random_grid = {'n_estimators': n_estimators,

'max_features': max_features,

'max_depth': max_depth,

'min_samples_split': min_samples_split,

'min_samples_leaf': min_samples_leaf,

'bootstrap': bootstrap}

In [11]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()

In [12]:
from sklearn.model_selection import RandomizedSearchCV
rf_random = RandomizedSearchCV(estimator = rf,param_distributions = random_grid,
               n_iter = 100, cv = 5, verbose=2, random_state=35, n_jobs = -1)

In [13]:
rf_random.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      120],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 3, 4],
                                        'min_samples_split': [2, 6, 10],
                                        'n_estimators': [5, 20, 50, 100]},
                   random_state=35, verbose=2)

In [14]:
print (f'Train Accuracy - : {rf_random.score(X_train,y_train):}')
print (f'Test Accuracy - : {rf_random.score(X_test,y_test):}')

Train Accuracy - : 0.7829823060381931
Test Accuracy - : 0.679870995566294


In [15]:
rf.fit(X_train,y_train)

RandomForestRegressor()

In [16]:
import joblib
joblib.dump(rf, 'price.pkl')

['price.pkl']

In [17]:
model = joblib.load('price.pkl')

In [18]:
x_pred=model.predict(X_train)
y_pred=model.predict(X_test)

In [19]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Absolute Percentage Error:', metrics.mean_absolute_percentage_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error :', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 1014.7519092977639
Mean Absolute Percentage Error: 0.3191534033212311
Mean Squared Error: 4873417.638083583
Root Mean Squared Error : 2207.5818530880306
